In [1]:
import sys,re, os, glob, json, string, pprint
import pandas as pd
import numpy as np 
from snowballstemmer import TurkishStemmer 
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from difflib import SequenceMatcher

word_size_threshold = 100
similarity_ratio = 0.70

## Read JSON

In [2]:

# similar(' 5607 sayılı Kanuna Aykırılık',' 5607 sayılı Kanuna Aykırılık')



path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json" 
files = glob.glob(path)

ictihats = []
suc_array = []

def similar(a, topTenCrimes):
    for crime in topTenCrimes:
        if SequenceMatcher(None, a, crime).ratio() > similarity_ratio:
            return crime
    return None
                
top_ten_crimes = []
with open("data.json", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    top_ten_crimes = list(map(lambda x: x[0], data[:10]))



for file in files:
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())
    key = jsonData["Suç"] if jsonData["Suç"] != '' else "undefined"
    similar_suc = similar(key, top_ten_crimes)
    
    key = similar_suc if similar_suc is not None else key
    suc = key if key in top_ten_crimes or key == 'undefined' else "other"

    new_ictihat = jsonData['ictihat'].strip()
    ictihats.append(new_ictihat)
    new_ictihat_ctr = new_ictihat.split()
    num_of_words = len(new_ictihat_ctr)
    suc_array.append(suc.strip())
        


 



## Create labels from dataset

In [3]:
# def has_key(dict, key): 
#     if key in dict.keys(): 
#         return 1 
#     return 0

# labels = {} 

# def similarity(a, b):
#     return SequenceMatcher(None, a, b).ratio()
    
# #similarity(' 5607 sayılı Kanuna muhalefet', ' 5607 Sayılı Kanuna Aykırılık' )
# def readLabels(fileName): 
#     f = open(file, "r", encoding='utf-8') 
#     jsonData = json.loads(f.read())
#     key = jsonData["Suç"] if jsonData["Suç"] != '' else "undefined"
#     if (not has_key(labels, key)):
#         labels[key] = 1    
#     else :
#         labels[key] = labels[key] + 1

# path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json" 
# files = glob.glob(path)

# for file in files: 
#     readLabels(file)


# items = list(map(lambda x: list(x), labels.items()))
# #print(type(items[0]))
# labels_array = sorted(items, key=lambda x: x[1], reverse=True)

# len_labels_array = len(labels_array)
# for i in range(0, len_labels_array):
    
#     for j in range(i, len_labels_array):
#         if labels_array[i][0] == labels_array[j][0] or labels_array[i][1] <= 1 or labels_array[j][1] <= 1 :
#             continue
#         if similarity(labels_array[i][0], labels_array[j][0]) > similarity_ratio:
#             labels_array[i][1] += labels_array[j][1]
#             labels_array[j][1] = 0

# array_final = (list(filter(lambda x: x[1] > 1 , labels_array)))

    
# array_final = sorted(array_final, key=lambda x: x[1], reverse=True)

    
# jsonString = json.dumps(array_final, indent = 4, ensure_ascii=False)
# jsonFile = open("data.json", "w", encoding='utf-8')
# jsonFile.write(jsonString)
# jsonFile.close()



## Preprocessing

In [4]:
ictihats_lower = []
for i in ictihats:
    i = i.lower()
    ictihats_lower.append(i)
    

ictihats_punctuation = []

file = open("stopwords.txt", "r", newline='', encoding='utf-8')
result = file.read()
stopwords = word_tokenize(result)

for ictihat in ictihats_lower:
    ictihat = ictihat.translate(str.maketrans("", "", string.punctuation))
    #print(ictihat)
    tokenized_words = ([word for word in ictihat.split() if word not in stopwords and len(word) > 1 and re.match("^[A-Z0-9a-zğüşöçİĞÜŞÖÇ]*$", word)])
    #print(tokenized_words)
    ictihats_punctuation.append(' '.join(tokenized_words[:word_size_threshold]))
    



# print(ictihats_punctuation[:10])   
# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#     preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# print(preprocessed_ictihats)
turkStem = TurkishStemmer()
for index, ictihat in enumerate(ictihats_punctuation,start=0):
    #print([turkStem.stemWord(word) for word in ictihat])
    ictihats_punctuation[index] = ' '.join([turkStem.stemWord(word) for word in ictihat.split()])
    
#print(ictihats_punctuation)

## write data into csv file

In [5]:
df = pd.DataFrame(columns = ['ictihats', 'sucs'])
df['ictihats'] = ictihats_punctuation
df['sucs'] = suc_array
csv_file_name = 'train_set_' + str(word_size_threshold) + '.csv'

df.to_csv(csv_file_name, encoding='utf-8', index=False)


## Split Dataset

In [6]:
# my_csv = pd.read_csv(filename)
# column = csv_file_name[]
# csv_file_name = 'train_set_' + str(word_size_threshold) + '.csv'
# my_csv = pd.read_csv(csv_file_name)
# #print(my_csv.columns)
# x = my_csv['ictihats']
# print(x.tolist())
# y = my_csv['sucs']
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=36)
#print(x_train)

x_train, x_test, y_train, y_test = train_test_split(ictihats_punctuation, 
                                                    suc_array, 
                                                    test_size=0.20,random_state=36)

## Vectorize

In [7]:
tfidfvectorizer = TfidfVectorizer(analyzer='word')
training_data = tfidfvectorizer.fit_transform(x_train)
testing_data = tfidfvectorizer.transform(x_test)


## Support Vector Machines (specifically linear SVM)

In [8]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(training_data, y_train)
y_pred = svclassifier.predict(testing_data)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 238    1    0    0    0    0    5    0    0   12    0]
 [  37   15    0    0    0    0    3    0    0   49    0]
 [   0    0   65    0    0    0    0    8    0   30    0]
 [   0    0    1   88    7    1    0    0    0   54    0]
 [   0    0    0   24   36    0    1    0    0   26    0]
 [   0    0    0   12    4   19    1    0    0   38    0]
 [   0    1    1    3    0    2  399    2    0   37    0]
 [   0    0    8    0    0    0    0   55    0   40    0]
 [   0    0    0    0    0    0    0    0  138    5    0]
 [   2    5   12   16    6    8   26   10   12 1391    3]
 [   2    2    0    0    0    0    3    0    2   16 2589]]
                                         precision    recall  f1-score   support

           5607 sayılı Kanuna muhalefet       0.85      0.93      0.89       256
           6136 sayılı Yasaya aykırılık       0.62      0.14      0.23       104
                                Hakaret       0.75      0.63      0.68       103
                              Hırsız

## Multinomial Naive Bayes

In [9]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)

# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
# print('Precision score: ', format(precision_score(y_test, predictions, average='weighted')))
# print('Recall score: ', format(recall_score(y_test, predictions ,average='weighted')))
# print('F1 score: ', format(f1_score(y_test, predictions ,average='weighted')))

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[ 118    0    0    0    0    0    0    0    0  131    7]
 [  22    0    0    0    0    0    1    0    0   77    4]
 [   0    0    0    0    0    0    0    0    0  102    1]
 [   0    0    0    0    0    0    0    0    0  144    7]
 [   0    0    0    0    0    0    0    0    0   82    5]
 [   0    0    0    0    0    0    2    0    0   68    4]
 [   0    0    0    0    0    0  218    0    0  205   22]
 [   0    0    0    0    0    0    0    0    0  103    0]
 [   0    0    0    0    0    0    0    0    1   79   63]
 [   0    0    0    0    0    0    6    0    0 1446   39]
 [   3    0    0    0    0    0    7    0    0  155 2449]]
                                         precision    recall  f1-score   support

           5607 sayılı Kanuna muhalefet       0.83      0.46      0.59       256
           6136 sayılı Yasaya aykırılık       0.00      0.00      0.00       104
                                Hakaret       0.00      0.00      0.00       103
                              Hırsız

C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

In [10]:
log_regres_classifier = LogisticRegression(random_state = 36)
log_regres_classifier.fit(training_data, y_train)
y_pred = log_regres_classifier.predict(testing_data)
cm = confusion_matrix(y_test, y_pred)


# print('Accuracy score: ', (accuracy_score(y_test, y_pred)))
# print('Precision score: ', (precision_score(y_test, y_pred, average='weighted')))
# print('Recall score: ', (recall_score(y_test, y_pred ,average='weighted')))
# print('F1 score: ', (f1_score(y_test, y_pred ,average='weighted')))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


C:\Users\onkol\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[ 231    1    0    0    0    0    6    0    0   17    1]
 [  34   16    0    0    0    0    3    0    0   51    0]
 [   0    0   58    0    0    0    0    8    0   37    0]
 [   0    0    0   80    8    0    0    0    0   62    1]
 [   0    0    0   23   34    2    1    0    0   25    2]
 [   0    0    0    7    3   14    1    0    0   49    0]
 [   0    0    2    2    0    0  397    2    0   41    1]
 [   0    0    8    0    0    0    0   49    0   45    1]
 [   0    0    0    0    0    0    0    0  137    6    0]
 [   4    3   11   13    7    8   26   10   12 1389    8]
 [   2    1    0    1    0    2    4    0    0   22 2582]]
                                         precision    recall  f1-score   support

           5607 sayılı Kanuna muhalefet       0.85      0.90      0.88       256
           6136 sayılı Yasaya aykırılık       0.76      0.15      0.26       104
                                Hakaret       0.73      0.56      0.64       103
                              Hırsız

In [11]:
#print(len(suc_array))
#print(ictihats_punctuation)
# index_list = []
# for i in range(0,100):
#     index_list.append(str(i))

# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#        preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# #print(preprocessed_ictihats)
# turkStem = TurkishStemmer()
# for index, ictihat in enumerate(preprocessed_ictihats,start=0):
#     preprocessed_ictihats[index] = [turkStem.stemWord(word) for word in ictihat]
    
#print(preprocessed_ictihats)

#tfidf_wm_array
#tfidf_wm_array.shape
# frequency_matrix = pd.DataFrame(tfidf_wm_array, columns=tfidfvectorizer.get_feature_names())
# from sklearn.model_selection import train_test_split
# y = np.array([0,1,2,3,4,5,6,7,8,9,10]) 


# naive_bayes = MultinomialNB()
# naive_bayes.fit(training_data, y_train)

# predictions = naive_bayes.predict(testing_data)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
# print('Precision score: ', format(precision_score(y_test, predictions,
#                                            average='macro')))
# print('Recall score: ', format(recall_score(y_test, predictions
#                                            ,average='macro')))
# print('F1 score: ', format(f1_score(y_test, predictions
#                                            ,average='macro')))

# print(confusion_matrix(y_test,predictions))


# from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(training_data, y_train)
# y_pred = svclassifier.predict(testing_data)

# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))



# frequency_matrix
# print(tfidfvectorizer.vocabulary_)
# print(tfidf_wm.toarray())
# tfidf_tokens = tfidfvectorizer.get_feature_names()
# df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = index_list,columns = tfidf_tokens)
# print("\nTD-IDF Vectorizer\n")
# print(df_tfidfvect)
# vectorizer.get_feature_names_out()
# ictihat_freq = []


# for i in preprocessed_ictihats:
#     count_freq = Counter(i)
#     ictihat_freq.append(count_freq)
    
# pprint.pprint(ictihat_freq)

